# 01_build_returns.ipynb

This notebook builds the data inputs for the Bayesian bandit portfolio project:

1. Download (or load) adjusted daily prices for a fixed universe of tickers using `yfinance`.
2. Compute daily log returns and save them in wide and long formats.
3. Construct a volatility–based active universe (top–K volatile names each day, with SPY always included if available).
4. Build several portfolio templates (equal–weight, momentum, low–volatility, and a static Markowitz portfolio) and save their daily return series for use in the bandit simulations.


## Setup: imports and directories

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from pathlib import Path

# Notebook is in bayes_bandit/notebooks
CWD = Path.cwd()
ROOT = CWD.parent

DATA_RAW = Path("../data_raw")
DATA_PROCESSED = Path("../data_processed")


for p in [DATA_RAW, DATA_PROCESSED]:
    p.mkdir(parents=True, exist_ok=True)

print("CWD:", CWD)
print("ROOT:", ROOT)
print("data_raw:", DATA_RAW)
print("data_processed:", DATA_PROCESSED)

CWD: c:\Users\nicka\dev\bayes_bandit\notebooks
ROOT: c:\Users\nicka\dev\bayes_bandit
data_raw: ..\data_raw
data_processed: ..\data_processed


## Download or load adjusted price data

We define a universe of liquid ETFs and large–cap stocks, then either:

- Load cached adjusted daily close prices from disk, or
- Download them from Yahoo Finance and cache the result as a parquet file.

Prices are adjusted using `auto_adjust=True` so that the `Close` column already reflects splits and dividends.

In [2]:
START_DATE = "2010-01-01"
END_DATE   = None  

UNIVERSE_TICKERS = [
    "SPY",  # broad market
    "QQQ",
    "TLT", "IEF", "HYG", "LQD",  # bonds / credit
    "GLD", "SLV",                # metals
    # large caps / sectors
    "AAPL", "MSFT", "GOOGL", "AMZN", "META", "NVDA",
    "JPM", "BAC", "GS", "WFC",
    "XOM", "CVX",
    "UNH", "JNJ",
    "HD", "PG", "KO",
    "PFE", "MRK",
    # extend as needed
]

UNIVERSE_TICKERS = sorted(set(UNIVERSE_TICKERS))
print("Universe size:", len(UNIVERSE_TICKERS))
print("First few tickers:", UNIVERSE_TICKERS[:10])

prices_path = DATA_RAW / "prices_adj_universe.parquet"

if prices_path.exists():
    print("Loading cached prices from:", prices_path)
    adj = pd.read_parquet(prices_path)
else:
    print("Downloading prices from Yahoo Finance...")
    data = yf.download(
        UNIVERSE_TICKERS,
        start=START_DATE,
        end=END_DATE,
        auto_adjust=True,   
        progress=False,
    )
    adj = data["Close"].copy()
    adj.columns.name = "ticker"
    adj.index.name = "date"

    # Remove NaN valjues
    adj = adj.dropna(axis=1, how="all")

    adj.to_parquet(prices_path)
    print("Saved prices to:", prices_path)

adj.head()

Universe size: 27
First few tickers: ['AAPL', 'AMZN', 'BAC', 'CVX', 'GLD', 'GOOGL', 'GS', 'HD', 'HYG', 'IEF']
Loading cached prices from: ..\data_raw\prices_adj_universe.parquet


ticker,AAPL,AMZN,BAC,CVX,GLD,GOOGL,GS,HD,HYG,IEF,...,NVDA,PFE,PG,QQQ,SLV,SPY,TLT,UNH,WFC,XOM
date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,6.418382,6.6950,12.232429,41.669128,109.800003,15.576997,130.641495,19.571789,35.435688,62.542007,...,0.423807,9.354956,38.417023,40.393551,17.230000,85.279190,56.984058,24.762829,18.026003,37.881802
2010-01-05,6.429481,6.7345,12.630036,41.964283,109.699997,15.508403,132.951248,19.715157,35.603783,62.816559,...,0.429995,9.221525,38.429596,40.393551,17.510000,85.504967,57.352074,24.723557,18.520864,38.029713
2010-01-06,6.327213,6.6125,12.778169,41.969559,111.510002,15.117455,131.532211,19.646885,35.695850,62.563118,...,0.432746,9.191875,38.247326,40.149910,17.860001,85.565140,56.584358,24.967024,18.547256,38.358406
2010-01-07,6.315513,6.5000,13.199171,41.811432,110.820000,14.765528,134.106049,19.878994,35.839951,62.563118,...,0.424265,9.157284,38.039879,40.176025,17.889999,85.926346,56.679508,25.925171,19.220259,38.237885
2010-01-08,6.357502,6.6760,13.082226,41.885201,111.370003,14.962370,131.569962,19.783417,35.896015,62.640572,...,0.425182,9.231411,37.989613,40.506691,18.150000,86.212280,56.654118,25.681702,19.042110,38.084488


## Compute daily log returns and build return panels

We compute one–day log returns

\[ r_{t} = \log P_{t} - \log P_{t-1} \]

and save them in:
- **Wide format**: one column per ticker.
- **Long format**: columns `['date', 'ticker', 'ret']`.

We also store a convenience panel parquet (`returns_panel.parquet`) for later use.

In [ ]:
# Wide logreturns
wide_ret = np.log(adj / adj.shift(1))
wide_ret.index.name = "date"

# remove NaNs
wide_ret = wide_ret.iloc[1:]

wide_ret_path = DATA_PROCESSED / "returns_wide_universe.parquet"
wide_ret.to_parquet(wide_ret_path)
print("Saved wide returns to:", wide_ret_path)

# Long format
returns_long = (
    wide_ret.stack()
    .reset_index()
    .rename(columns={0: "ret"})
)

returns_long = returns_long.dropna(subset=["ret"])

returns_long_path = DATA_PROCESSED / "returns_long_universe.parquet"
returns_long.to_parquet(returns_long_path)
print("Saved long returns to:", returns_long_path)

returns_long.head()

Saved wide returns to: ..\data_processed\returns_wide_universe.parquet
Saved long returns to: ..\data_processed\returns_long_universe.parquet


,date,ticker,ret
0,2010-01-05,AAPL,0.001728
1,2010-01-05,AMZN,0.005883
2,2010-01-05,BAC,0.031987
3,2010-01-05,CVX,0.007058
4,2010-01-05,GLD,-0.000911


## Descriptive statistics by ticker

As a basic sanity check, we compute distributional summaries of daily returns for each ticker.

In [4]:
summary = returns_long.groupby("ticker")["ret"].agg(
    mean="mean",
    std="std",
    skew=lambda x: x.skew(),
    kurtosis=lambda x: x.kurt(),
)

summary

,mean,std,skew,kurtosis
ticker,,,,
AAPL,0.000941,0.017772,-0.151399,6.121489
AMZN,0.000882,0.020664,0.031863,5.953125
BAC,0.000370,0.020993,-0.317604,10.154065
CVX,0.000320,0.016883,-0.927561,26.046087
GLD,0.000314,0.009966,-0.501302,4.746034
GOOGL,0.000755,0.017396,0.125003,7.085535
GS,0.000469,0.018162,-0.254088,8.303651
HD,0.000723,0.014711,-0.989131,18.273950
HYG,0.000206,0.005257,-0.252150,19.150714


In [5]:
panel_path = DATA_PROCESSED / "returns_panel.parquet"
returns_long.to_parquet(panel_path, index=False)

print("Saved returns panel to:", panel_path)
print("Shape:", returns_long.shape)

test = pd.read_parquet(panel_path)
print(test.head())
print(test.dtypes)

Saved returns panel to: ..\data_processed\returns_panel.parquet
Shape: (107563, 3)
        date ticker       ret
0 2010-01-05   AAPL  0.001728
1 2010-01-05   AMZN  0.005883
2 2010-01-05    BAC  0.031987
3 2010-01-05    CVX  0.007058
4 2010-01-05    GLD -0.000911
date      datetime64[ns]
ticker            object
ret              float64
dtype: object


## Build volatility–based active universe (top–K + SPY anchor)

We construct an **active universe** for each trading day:

1. Compute rolling realised volatility over a lookback window.
2. For each date, select the top–K tickers by volatility.
3. Always include `SPY` if it is available on that date.
4. Save the resulting panel of active memberships as `active_universe_top{K}_vol_plus_SPY.parquet`.

This will be the pool of candidate assets for portfolio construction and bandit policies.

In [ ]:
wide_ret = pd.read_parquet(DATA_PROCESSED / "returns_wide_universe.parquet")

LOOKBACK = 60      # trading days
TOP_K = 20         # active tickers per day (excluding SPY)
ANCHOR_TICKER = "SPY"

# Rolling stdev, then shift by 1 day so that vol at date t uses data up to t-1
daily_vol = wide_ret.rolling(window=LOOKBACK, min_periods=LOOKBACK).std().shift(1)

rows = []

for date, row in daily_vol.iterrows():
    vols = row.dropna()
    if vols.empty:
        continue
    vols_sorted = vols.sort_values(ascending=False)
    top_tickers = list(vols_sorted.head(TOP_K).index)

    if ANCHOR_TICKER in vols.index and ANCHOR_TICKER not in top_tickers:
        top_tickers.append(ANCHOR_TICKER)

    for ticker in top_tickers:
        rows.append((date, ticker))

active_universe = pd.DataFrame(rows, columns=["date", "ticker"])
active_universe["active"] = True

active_path = DATA_PROCESSED / f"active_universe_top{TOP_K}_vol_plus_{ANCHOR_TICKER}.parquet"
active_universe.to_parquet(active_path)

print("Saved active universe to:", active_path)
active_universe.head()

Saved active universe to: ..\data_processed\active_universe_top20_vol_plus_SPY.parquet


,date,ticker,active
0,2010-04-01,NVDA,True
1,2010-04-01,BAC,True
2,2010-04-01,UNH,True
3,2010-04-01,SLV,True
4,2010-04-01,AMZN,True


## Construct portfolio templates

Using the active universe and feature estimates from returns, we build daily return series for several portfolio templates:

- **EW_active**: Equal–weight all active tickers each day.
- **EW_top5vol**: Equal–weight the 5 most volatile active tickers.
- **MOM_top5**: Equal–weight the 5 highest momentum active tickers (past 60 trading days of log returns).
- **LOWVOL_top5**: Equal–weight the 5 lowest volatility active tickers.
- **Markowitz_static**: A static tangency–style Markowitz portfolio estimated on an initial training window and then held out–of–sample.

We then save the combined panel of portfolio returns as `portfolio_returns_dynamic.parquet`. This will be the input to the bandit environment in `02_bandit_sim.ipynb`.

In [ ]:
LOOKBACK = 60       # used for vol and momentum
TOP_K    = 20       # active universe size
MOM_K    = 5        # number of high-momentum names
LOWVOL_K = 5        # number of low-volatility names

wide_ret = pd.read_parquet(DATA_PROCESSED / "returns_wide_universe.parquet")

# Rolling vol
daily_vol = wide_ret.rolling(window=LOOKBACK, min_periods=LOOKBACK).std().shift(1)

# Rolling momentums
mom_raw = wide_ret.rolling(window=LOOKBACK, min_periods=LOOKBACK).sum().shift(1)

# Data loading
active_universe = pd.read_parquet(
    DATA_PROCESSED / f"active_universe_top{TOP_K}_vol_plus_SPY.parquet"
)

active_map = (
    active_universe
    .groupby("date")["ticker"]
    .apply(list)
    .to_dict()
)

portfolio_rows = []

for date in wide_ret.index:
    if date not in active_map:
        continue

    active_tickers = active_map[date]
    active_tickers = [t for t in active_tickers if t in wide_ret.columns]
    if len(active_tickers) == 0:
        continue

    ret_t = wide_ret.loc[date, active_tickers]

    # EW_active: equal-weight all active tickers ---
    w_e = np.ones(len(active_tickers)) / len(active_tickers)
    ret_e = float(np.dot(w_e, ret_t.values))
    portfolio_rows.append((date, "EW_active", ret_e))

    # EW_top5vol: equal-weight of top 5 vol among active tickers ---
    vol_row = daily_vol.loc[date, active_tickers].dropna()
    if len(vol_row) >= 5:
        top5_vol = vol_row.sort_values(ascending=False).head(5).index
        ret_top5vol = wide_ret.loc[date, top5_vol]
        w5 = np.ones(len(top5_vol)) / len(top5_vol)
        ret_5_vol = float(np.dot(w5, ret_top5vol.values))
        portfolio_rows.append((date, "EW_top5vol", ret_5_vol))

    # MOM_top5: top 5 momentum names among active tickers ---
    mom_row = mom_raw.loc[date, active_tickers].dropna()
    if len(mom_row) >= MOM_K:
        top_mom = mom_row.sort_values(ascending=False).head(MOM_K).index
        ret_mom = wide_ret.loc[date, top_mom]
        w_mom = np.ones(len(top_mom)) / len(top_mom)
        ret_mom_port = float(np.dot(w_mom, ret_mom.values))
        portfolio_rows.append((date, "MOM_top5", ret_mom_port))

    # LOWVOL_top5: 5 lowest-vol names among active tickers ---
    lowvol_row = vol_row  # already volatility of active tickers
    if len(lowvol_row) >= LOWVOL_K:
        low5 = lowvol_row.sort_values(ascending=True).head(LOWVOL_K).index
        ret_low5 = wide_ret.loc[date, low5]
        w_low5 = np.ones(len(low5)) / len(low5)
        ret_lowvol_port = float(np.dot(w_low5, ret_low5.values))
        portfolio_rows.append((date, "LOWVOL_top5", ret_lowvol_port))

# Static Markowitz portfolio (Markowitz_static)

TRAIN_DAYS = 252 * 5   # ~5 years of daily data

if wide_ret.shape[0] > TRAIN_DAYS:
    train = wide_ret.iloc[:TRAIN_DAYS].dropna(axis=1, how="any")
    train_cols = train.columns.tolist()

    mu_hat = train.mean()            # (n_assets,)
    Sigma_hat = train.cov()          # (n_assets x n_assets)

    Sigma_inv = np.linalg.pinv(Sigma_hat.values)
    mu_vec = mu_hat.values.reshape(-1, 1)
    w_unnorm = Sigma_inv @ mu_vec    
    w_unnorm = w_unnorm.flatten()

    w_unnorm = np.clip(w_unnorm, 0.0, None)
    if w_unnorm.sum() == 0.0:
        w_unnorm = np.ones_like(w_unnorm)

    w_star = w_unnorm / w_unnorm.sum()

    markowitz_tickers = train_cols
    w_star_series = pd.Series(w_star, index=markowitz_tickers)
    wide_mark = wide_ret[markowitz_tickers].dropna(how="any", axis=0)
    mark_ret_series = (wide_mark @ w_star_series).dropna()

    for d, r in mark_ret_series.items():
        portfolio_rows.append((d, "Markowitz_static", float(r)))
else:
    print("Not enough data for Markowitz_static; skipping that template.")

port_returns = pd.DataFrame(portfolio_rows, columns=["date", "portfolio", "ret"])
port_returns = port_returns.sort_values("date")

port_ret_path = DATA_PROCESSED / "portfolio_returns_dynamic.parquet"
port_returns.to_parquet(port_ret_path)

print("Saved dynamic portfolio returns to:", port_ret_path)
print(port_returns["portfolio"].value_counts())

port_returns.head()

Saved dynamic portfolio returns to: ..\data_processed\portfolio_returns_dynamic.parquet
portfolio
Markowitz_static    4006
EW_active           3946
EW_top5vol          3946
MOM_top5            3946
LOWVOL_top5         3946
Name: count, dtype: int64


,date,portfolio,ret
15784,2010-01-05,Markowitz_static,0.003374
15785,2010-01-06,Markowitz_static,-0.002281
15786,2010-01-07,Markowitz_static,0.004022
15787,2010-01-08,Markowitz_static,0.000735
15788,2010-01-11,Markowitz_static,-0.002843
